In [22]:
# == importando bibliotecas == 

import pandas as pd
import numpy as np
import re

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

In [23]:
# == ler a base de artigos == 

#artigos = pd.read_parquet('../../dataset/processed/artigos_de_partidos/artigos_partidos.parquet')
artigos = pd.read_csv('../../predicao-final/noticias-teste-oeste-cart_capital.csv')
artigos.head()

,Conteudo
0,"\n A presidente do PT, Gleisi H..."
1,\n O combate à corrupção tem si...
2,"\n Contudo, não apenas dessa, p..."
3,\n Deixando os aspectos metafís...
4,\n Modelos não sabem simular o S...


In [24]:
# == tornar os artigos em letras minúsculas == 

artigos.Conteudo = artigos.Conteudo.str.lower()

In [25]:
# == remover artigos contendo links == 

artigos[artigos.Conteudo.str.contains('http')].Partido.value_counts()
artigos = artigos[~artigos.Conteudo.str.contains('http')]

AttributeError: 'DataFrame' object has no attribute 'Partido'

In [26]:
# == todas as sentenças após o penúltimo "." a fim de eliminar ruídos que não fazem parte do artigo == 

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ''.join(x.split('.')[:-2])
)

In [27]:
# == mascarar nome de partido == 

partidos = ['novo', 'pcb', 'pdt', 'pl', 'pp', 'psol', 'pstu', 'mdb', 'pcdob', 'psb', 'pt', 'pv', 'rede', 'união brasil']
for partido in partidos:
    artigos.Conteudo = artigos.Conteudo.str.replace(f' {partido} ', ' ')

In [28]:
# == remover todas as palavras contendo caracteres especiais ==

artigos.Conteudo = artigos.Conteudo.apply(
    lambda x : ' '.join(word for word in x.split(' ') if word.isalnum())
)

In [29]:
# == normalização por unicode e encode ascii ==

artigos.Conteudo = artigos.Conteudo.str.lower()\
                                   .str.normalize('NFKD')\
                                   .str.encode('ascii', errors='ignore')\
                                   .str.decode('utf-8')

In [30]:
# == remover pontuações e números == 

artigos.Conteudo = artigos.Conteudo.str.replace('[^a-zA-Z]', ' ')

In [31]:
# == remover double space == 

for n in range(30):
    artigos.Conteudo = artigos.Conteudo.str.replace('  ', ' ')

In [32]:
# == analisar a contagem média de palavras por partido == 

copy = artigos.copy()
copy['Contagem'] = copy.Conteudo.str.count(' ')
copy.groupby(
    by=['Vies', 'Partido'],
).agg(
    ContagemPalavras = ('Contagem', 'mean'), 
    ContagemArtigos = ('Contagem', 'count')
)

KeyError: 'Vies'

In [33]:
# == salvar o df tratado == 

#artigos.to_parquet('../../dataset/processed/artigos_tratados/artigos_tratados.parquet')
artigos.to_csv('../../predicao-final/noticias-teste-oeste-cart_capital-tratado.csv')

In [37]:
artigos

,Conteudo
0,a presidente do gleisi disse a militantes do p...
1,o combate a corrupcao tem sido uma fraqueza no...
2,nao apenas pois as reunioes e cerimonias paral...
3,deixando os aspectos metafisicos de lado por u...
4,modelos nao sabem simular o os os o os ciclos ...
5,o jornal chamou michael o arquiteto do f3 fada...
6,o que mann e o outro pesquisador de andrew des...
7,as nuvens especialmente as do tipo cumulus ref...
8,nao faz o menor sentido o que mann e dessler f...
9,vimos novamente o professor john francis laure...
